## Adiabatic quantum computing

Suppose we need to find the ground state of a Hamiltonian $H_1$ that we can prepare. To do that, we first prepare the Hamiltonian $H_0$, then slowly change it into the target Hamitlonian. Adiabatic theorem states that under these conditions the system remains in the ground state. 

More specifically, the time-dependent Hamiltonian we will consider in this example is the following:

$$
H(t) = (1 - s(t))H_0 + s(t) H_1,
$$

where $s(t)$ depends on our strategy of adiabatic change. In the adiabatic theorem, "slowly" means with respect to the energy scale of the Hamiltonian. That is, if the gap is small, we should change the parameter slowly, if the gap is large, we can get away with changing it faster.

The overall evolution of the quantum state is governed by Schrödinger equation:

$$
\frac{d \psi(t)}{dt} = -i H(t) \psi(t),
$$
where $\psi$ is the state vector.

In this example, the target Hamiltonians will be the Ising model and the Hamiltonians created by embedding 3-SAT instances.

In [ ]:
import numpy as np
import scipy.linalg as SPLA
from scipy.integrate import solve_ivp
from functools import reduce
import matplotlib.pyplot as plt
from collections import deque

### Define the initial and target Hamiltonians

In [ ]:
X = np.array([[0, 1], [1, 0]], dtype=np.complex64)
Y = np.array([[0, -1j], [1j, 0]])
Z = np.diag([1, -1])
I = np.eye(2)

#### Ising model

The Ising model with transverse field is a Hamiltonian that is known to have an entangled ground state at nonzero field:

$$
H = J \sum Z_i Z_{i+1} + h \sum X_i.
$$

We assume that the boundary conditions are periodic.

In [ ]:
def ising_model(n_spins, J, hx):
    '''Ising model with transverse field.'''
    pass

#### SAT embedding

Any instance of K-SAT problem can be embedded in a Hamiltonian. Each clause adds a term that penalizes all states in computational basis that violate this clause. The question of "Is this Boolean expression satisfiable?" turns into "Does this Hamiltonian have the ground state energy of zero or $\geq 1$, promised that one of these is the case?"

To do this we need to transform every clause to operator form to make penalty function.
$$(x_{i} \lor \overline{x}_{j} \lor x_{k}) \mapsto 
\vert 0\rangle\!\langle 0\vert_i \otimes \vert 1\rangle\!\langle 1\vert_j \otimes \vert 0\rangle\!\langle 0\vert_k
$$
and then sum all these operators. 
<br/>
$$
P_0=\vert 0\rangle\!\langle 0\vert,\\
P_1=\vert 1\rangle\!\langle 1\vert.
$$
Since the 3-SAT problem consists of n variables then the clause operator has the following form
$$
(x_{i} \lor \overline{x}_{j} \lor x_{k}) \mapsto I_1\otimes\ldots\otimes I_{i-1} \otimes \vert 0\rangle\!\langle 0\vert_i \otimes I_{i+1}\cdots \otimes I_{j-1}\otimes \vert 1\rangle\!\langle 1\vert_j \otimes I_{j+1}\cdots \otimes I_{k-1} \otimes \vert 0\rangle\!\langle 0\vert_k \otimes I_{k+1}\cdots \otimes I_{n}
$$

In [ ]:
def random_clause(num_variables, k=3):
    '''Generate a random clause for k-SAT. Returns a list of three
    integers
    '''
    clause = np.random.choice(num_variables, k, replace=False) + 1
    clause = [c * ((-1)**np.random.randint(2)) for c in clause]
    return clause

def random_SAT_instance(num_variables, num_clauses, k=3):
    '''
    Generate a random instance of K-SAT
    '''
    return [random_clause(num_variables, k) for i in range(num_clauses)]

def make_SAT_Hamiltonian(sat_instance, num_variables):
    '''Creates a Hamiltonian from a SAT instance. '''    
    pass

#### Putting it together and defining the initial state

We define an "easy" Hamiltonian to be $H_0 = - \sum X_i$. The ground state of this Hamiltonian is 

$$
| + \rangle = \frac{1}{2^n} (1 \dots 1)^T.
$$

In [ ]:
n_qubits = 6

ham_term = deque([X] + [I] * (n_qubits - 1))

H_0 = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex64)

for i in range(n_qubits):
    H_0 += (-0.5) * reduce(np.kron, ham_term)
    ham_term.rotate(1)

In [ ]:
# Initial state is a Kronecker product of |+> = 1/ sqrt(2) * (|0> + |1>)
total_plus_state = np.ones(2**n_qubits, dtype=np.complex64) / (2**n_qubits)**0.5

### Define the system of differential equations

In [ ]:
def s_linear(t, alpha):
    '''Adiabatic evolution strategy'''
    if t < 0:
        raise ValueError('Enter positive t')
    elif t > 1 / alpha:
        return 1
    else:
        return t * alpha      
    
def get_state_energy(s, t, y):
    '''Energy of the state measured at time t'''
    H = H_total(H_0, H_1, s(t))
    return y.conj() @ H @ y

def H_total(H_0, H_1, s_curr):
    '''Hamiltonian at the parameter value s_curr'''
    return (1 - s_curr) * H_0 + s_curr * H_1

def make_f(H_0, H_1, s):
    '''Create a system of differential equations'''
    def f(t, y):
        #H = (1 - s(t)) * H_0 + s(t) * H_1
        H = H_total(H_0, H_1, s(t))
        return -1j * H @ y
    return f

The parameter `alpha` controls the speed of changing the Hamiltonian. The smaller it is, the closer should the approximated energy be to the ground state energy. Try changing it or also try different trajectories of `s(t)` altogether.

At this point, also define `H_1` as a Hamiltonian of your choice.

In [ ]:
alpha = 0.2

def s(t):
    '''Simple strategy of changing the Hamiltonian'''
    return s_linear(t, alpha)

f = make_f(H_0, H_1, s)

### Run the solver

In [ ]:
sol = solve_ivp(f, [0, 1 / alpha], total_plus_state, max_step=0.2)

In [ ]:
exact_energies = []
energies = []
for i, t in enumerate(sol.t):
    energies.append(get_state_energy(s, t, sol.y[:, i]).real)
    H = H_total(H_0, H_1, s(t))
    w, v = np.linalg.eigh(H)
    exact_energies.append(w[0])

In [ ]:
plt.scatter(sol.t, energies, label='AQC')
plt.plot(sol.t, exact_energies, label='Exact GS')
plt.xlabel('t')
plt.ylabel('Energy')
plt.legend()
plt.show()